##  2. Inclusive Ratings by score and year
<body> <p  style = "font-family:georgia,garamond,serif;font-size:16px;text-align:justify"></body> 
We store in our PostgreSQL database the residual of the regression (inclusive rating) for each score on yearly basis according to equation 1 of the research.

In [1]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.express as px
from sqlalchemy import create_engine 
import ast

In [2]:
credentials = ast.literal_eval(open(r"C:\Users\pvilas\OneDrive - unizar.es\Python Investigación\credentials.txt", "r").read())
engine = create_engine(credentials['sqlalchemy'][0]+ 'ESGRefinitiv')

df_a22 = pd.read_sql("SELECT * FROM a22_yearly_esg_mv_percentiles WHERE year BETWEEN 2010 AND 2019", engine)

conv_ratings = ["esg_score_average", "environment_pillar_average", "social_pillar_average", "governance_pillar_average"]
years = set(df_a22['year'])

# the name of our Inclusive ratings
incl_ratings = ['overall_inclusive', 'environmental_inclusive', 'social_inclusive', 'governance_inclusive']

# Nª of companies by year & country
df_a22 = df_a22.groupby(['year', 'ds_country'])[['esg_score_average']].count().rename(columns={
    'esg_score_average':'n_firms'}).reset_index().merge(df_a22, on=['year', 'ds_country'])

In [3]:
df_a22.dropna(subset=['ds_country','industry','percent_rank_mv_usd']+conv_ratings, inplace=True)
# companies belonging to a country with less than 4 ESG-rated companies are excluded
df_a22 = df_a22[df_a22['n_firms']>=4].reset_index(drop=True) 
table_inclusive = pd.DataFrame() # pAPER tABLE 2

for conv_rating, incl_rating in zip(conv_ratings, incl_ratings):
    
    df_resids =pd.DataFrame()
    for year in (years):
        res = smf.ols(('%s ~ percent_rank_mv_usd + C(ds_country) + C(industry)' % conv_rating), 
                      data=df_a22[df_a22['year']== year]).fit()
        df_resids= pd.concat([df_resids, res.resid])
        # Table Info
        table_inclusive.loc[incl_rating, int(year)] = '{:.3f}'.format(res.rsquared.round(3))
        if incl_rating == "governance_inclusive":
            table_inclusive.loc['#_countries', year] = str(len(df_a22[df_a22['year']== year]['ds_country'].unique()))
            table_inclusive.loc['observations', year] = str(int(res.nobs))
            
    # Introduce the resids
    df_a22[incl_rating] = df_resids
                   
df_a22.to_sql("b21_isp_ssp_esg_mv", engine, if_exists='replace', chunksize=100000, index=False)
table_inclusive.sort_index(axis=1).to_excel(
    r'C:\Users\pvilas\OneDrive - unizar.es\Tesis\2022 ISP\3. Tables & Figures\3. Definitive\Table 2.xlsx') 
table_inclusive.sort_index(axis=1)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
overall_inclusive,0.381,0.371,0.348,0.356,0.347,0.366,0.396,0.405,0.393,0.418
environmental_inclusive,0.412,0.399,0.380,0.382,0.383,0.423,0.468,0.481,0.452,0.474
social_inclusive,0.390,0.382,0.361,0.368,0.360,0.357,0.367,0.387,0.382,0.412
governance_inclusive,0.120,0.126,0.118,0.119,0.101,0.113,0.130,0.128,0.131,0.140
#_countries,44,45,45,45,47,50,52,54,54,54
observations,3815,3947,4003,4116,4372,5240,6006,6543,7377,8070
